In [68]:
import urllib2, re, os
import bs4
import pandas as pd
from tqdm import tqdm_notebook

In [5]:
SVO_BASE_URL = 'http://svo2.cab.inta-csic.es/svo/theory/fps'

In [48]:
def generate_facilities_list():
    svo_html = urllib2.urlopen(SVO_BASE_URL).read()
    svo_soup = bs4.BeautifulSoup(svo_html)
    observatory_links = svo_soup.findAll('a', attrs={'href':re.compile('gname')})
    observatory_list = pd.DataFrame(index=range(len(observatory_links)), columns=['facility', 'url'])
    for i, obs_link in enumerate(observatory_links):
        observatory_list.loc[i] = obs_link.text, obs_link.attrs['href']
    return observatory_list

def generate_instruments_list(observatory_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    return observatory_soup.findAll('a', attrs={'href':re.compile('index\.php\?mode=browse&gname=.+?&gname2=.+?')})
#    filter_soup = observatory_soup.findAll('table')[8]
#    filter_table = pd.read_html(str(filter_soup), header=0)[0]
#    return filter_table

def generate_filter_list(instrument_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    filter_soup = observatory_soup.findAll('table')[8]
    filter_table = pd.read_html(str(filter_soup), header=0)[0]
    return filter_table

In [47]:
facilities_list = generate_facilities_list()

In [52]:
facility.url

'index.php?mode=browse&gname=2MASS'

In [70]:
tqdm_notebook?

In [71]:
all_filters = None
for i, facility in tqdm_notebook(facilities_list.iterrows(), desc='Facilities', total=len(facilities_list)):
    #print "Working on {0}".format(facility.facility)
    instruments = generate_instruments_list(os.path.join(SVO_BASE_URL, facility.url))
    if len(instruments) == 0:
        #print "{0} does not have separate instruments".format(facility.facility)
        filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, facility.url))
        if all_filters is None:
            all_filters = filter_list
        else:
            all_filters = all_filters.append(filter_list)
    else:
        for instrument in tqdm_notebook(instruments, desc='Instruments', leave=False):
            #print "Working on {0}".format(instrument.text)
            filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, instrument.attrs['href']))
            if all_filters is None:
                all_filters = filter_list
            else:
                all_filters = all_filters.append(filter_list)


In [74]:
all_filters.to_hdf('svo_all_filters.h5', 'all_filters')

/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/pandas/io/pytables.py:270: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  f(store)
/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/pandas/io/pytables.py:270: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_items] [items->None]

  f(store)
/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/pandas/io/pytables.py:270: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_items] [items->None]

  f(store)
/Users/wkerzend/anaconda3/envs/starkit/lib/python2.7/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as